In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

SyntaxError: invalid syntax (constants.py, line 184)

In [ ]:
import wandb
run_id = wandb.util.generate_id(); run_id

In [ ]:
wandb.init(id='19fc9tj5', project="carla", resume="allow")

In [ ]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

In [ ]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

In [ ]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [ ]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

In [ ]:
%%time
#rw_dataloader = RealWorldDataloader()

In [ ]:
set_lr(3e-4)

In [ ]:
model_stem = "11.29"
START_E = 15

In [ ]:
n_epochs = 10000
log_wandb = True

for e in range(START_E, n_epochs):
    
    lr = get_lr()
    opt.param_groups[0]['lr'] = lr if lr!=42 else (6e-4 if e < 138 else 3e-4)

    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=2560)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

In [12]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m.torch")

Error in callback <function _WandbInit._resume_backend at 0x7fb3b40e8ee0> (for pre_run_cell):


[autoreload of constants failed: Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 839, in exec_module
  File "<frozen importlib._bootstrap_external>", line 976, in get_code
  File "<frozen importlib._bootstrap_external>", line 906, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_r

Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x7fb388ffb700> (for post_run_cell):


Exception: The wandb backend process has shutdown

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 167, in check_status
    status_response = self._interface.communicate_stop_status()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 109, in communicate_stop_status
    resp = self._communicate_stop_status(status)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface_queue.py", line 401, in _communicate_stop_status
    resp = self._communicate(req, local=True)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/wandb/sdk/interface/interface_queue.py", line 232

In [13]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
